<a href="https://colab.research.google.com/github/sahil-bhatia-iitbhu/deepseek-from-scratch/blob/main/04_tokenizer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Input the raw data and tokenize: The Verdict Story

In [18]:
import os
import requests

if not os.path.exists("the-verdict.txt"):
  url = (
      "https://raw.githubusercontent.com/sahil-bhatia-iitbhu/deepseek-from-scratch/main/04_input_the-verdict.txt"
  )
  file_path = "the-verdict.txt"

  response = requests.get(url, timeout=30)
  response.raise_for_status()
  with open(file_path, "wb") as f:
      f.write(response.content)
  # After writing, get raw_text from response.content
  raw_text = response.content.decode('utf-8')
else:
  # If the file already exists, read it
  with open("the-verdict.txt", "r", encoding='utf-8') as f:
      raw_text = f.read()

print(f"Total number of characters in the text file:\n{len(raw_text)}")
print(" ")
print(f"Sample Raw Text:\n{raw_text[:99]}")

Total number of characters in the text file:
20479
 
Sample Raw Text:
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [19]:
# let's first understand the tool for Step 1: Tokenizing the Text
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [20]:
# If we want to split on not only whitespaces but on different other types of period and punctuation

text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [22]:
# Now let's tokenize our raw_text

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(f"Token Sample:\n{preprocessed[:30]}")
print(" ")
print(f"Total number of tokens:\n{len(preprocessed)}")

Token Sample:
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
 
Total number of tokens:
4690


### Generate Token IDs and Create *class* for both encoders and decoders

In [26]:
unique_words = set(preprocessed)
print(f"Total number of unique words:\n{len(unique_words)}\n")

all_words = sorted(unique_words)
print(f"Sample sorted unique words:\n{all_words[:29]}\n")

vocab_size = len(all_words)
print(f"Vocabulary Size:\n{vocab_size}")

Total number of unique words:
1130

Sample sorted unique words:
['!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At', 'Be', 'Begin', 'Burlington', 'But', 'By', 'Carlo', 'Chicago', 'Claude', 'Come', 'Croft']

Vocabulary Size:
1130


In [29]:
# now, let's assign unique integer to each token, means generating token IDs

vocab = {token:integer for integer,token in enumerate(all_words)}

In [32]:
print("Sample Token ID - Token Space\n")
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break

Sample Token ID - Token Space

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [35]:
## We would create a tokenizer class for both encoding and decoding
## encoding converts the input to token IDs present in the vocabulary
## decoding converts the token IDs back to text

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [37]:
# setup the class with vocabulary
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [39]:
# hope we get the same text back from the IDs
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

### Adding Special Context Token

Some tokenizers use special tokens to help the LLM with additional context

Some of these special tokens are:

- [BOS] (beginning of sequence) marks the beginning of text
- [EOS] (end of sequence) marks where the text ends (this is usually used to concatenate multiple unrelated texts, e.g., two different Wikipedia articles or two different books, and so on)
- [PAD] (padding) if we train LLMs with a batch size greater than 1 (we may include multiple texts with different lengths; with the padding token we pad the shorter texts to the longest length so that all texts have an equal length)
- [UNK] to represent words that are not included in the vocabulary

Note that GPT/Deepseek does not need any of these tokens mentioned above but only uses an <|endoftext|> token to reduce complexity

The <|endoftext|> is analogous to the [EOS] token mentioned above

GPT/Deepseek also uses the <|endoftext|> for padding (since we typically use a mask when training on batched inputs, we would not attend padded tokens anyways, so it does not matter what these tokens are)

GPT/Deepseek does not use an <UNK> token for out-of-vocabulary words; instead, GPT-2 uses a byte-pair encoding (BPE) tokenizer, which breaks down words into subword units which we will discuss in a later section

In [41]:
# Lets first hit a text with unknown word in out v1 tokenizer

tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

# we would get an error as "Hello" is not present in the vocabulary

KeyError: 'Hello'

In [44]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [45]:
print(f"Updated length of vocabulary: {len(vocab.items())}")

Updated length of vocabulary: 1132


In [47]:
# The appended tokens are added at the end of the vocabulary

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [48]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [49]:
# Now, let's test the V2 of tokenizer

tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [50]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [51]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'